# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loguru import logger
from pprint import pprint
import json

import sys
sys.path.insert(0, '..')

In [3]:
import nest_asyncio
from tqdm.asyncio import tqdm_asyncio

nest_asyncio.apply()

# Arguments

In [4]:
from src.run.args import RunInputArgs

ARGS = RunInputArgs(
    EXPERIMENT_NAME="Chain Frost - LlamaIndex Blog QnA Chatbot",
    RUN_NAME="010_remove_title_extractor",
    RUN_DESCRIPTION="""
# Refactor

## Changelog
### Compares to exp_009
- Remove Title Extractor
""",
    TESTING=False,
    DEBUG=False,
    LOG_TO_MLFLOW=True,
    OBSERVABILITY=True,
    RECREATE_INDEX=True,
    RECREATE_RETRIEVAL_EVAL_DATASET=True,
    RECREATE_RESPONSE_EVAL_DATASET=True,
)

ARGS

{
  "EXPERIMENT_NAME": "Chain Frost - LlamaIndex Blog QnA Chatbot",
  "RUN_NAME": "010_remove_title_extractor",
  "RUN_DESCRIPTION": "\n# Refactor\n\n## Changelog\n### Compares to exp_009\n- Remove Title Extractor\n",
  "TESTING": false,
  "DEBUG": false,
  "OBSERVABILITY": true,
  "LOG_TO_MLFLOW": true,
  "RECREATE_INDEX": true,
  "RECREATE_RETRIEVAL_EVAL_DATASET": true,
  "RECREATE_RESPONSE_EVAL_DATASET": true
}

# Load config

In [5]:
from src.run.cfg import RunConfig

In [6]:
cfg = RunConfig()
cfg.init(ARGS)

2024-07-26 14:53:19.335 | INFO     | src.run.cfg:init:93 - Starting Observability server with Phoenix...
I0000 00:00:1721980404.082361 1358983 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


2024-07-26 14:53:24.689 | INFO     | src.run.cfg:init:111 - Setting up MLflow experiment Chain Frost - LlamaIndex Blog QnA Chatbot - run 010_remove_title_extractor...
2024-07-26 14:53:25.062 | INFO     | src.run.cfg:init:120 - Notebook-generated artifacts are persisted at data/010_remove_title_extractor
2024-07-26 14:53:25.063 | INFO     | src.run.cfg:init:126 - ARGS.RECREATE_INDEX=True -> Overwriting db_collection and nodes_persist_fp...


In [7]:
cfg

/home/dvquys/frostmourne/study/vietai-genai03/assignment1/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{
  "args": {
    "EXPERIMENT_NAME": "Chain Frost - LlamaIndex Blog QnA Chatbot",
    "RUN_NAME": "010_remove_title_extractor",
    "RUN_DESCRIPTION": "\n# Refactor\n\n## Changelog\n### Compares to exp_009\n- Remove Title Extractor\n",
    "TESTING": false,
    "DEBUG": false,
    "OBSERVABILITY": true,
    "LOG_TO_MLFLOW": true,
    "RECREATE_INDEX": true,
    "RECREATE_RETRIEVAL_EVAL_DATASET": true,
    "RECREATE_RESPONSE_EVAL_DATASET": true
  },
  "db_collection": "huggingface__BAAI_bge_large_en_v1_5__010_remove_title_extractor",
  "nodes_persist_fp": "data/010_remove_title_extractor/nodes.pkl",
  "notebook_cache_dp": "data/010_remove_title_extractor",
  "data_fp": "../crawl_llamaindex_blog/data/blogs-v2.json",
  "llm_cfg": {
    "llm_provider": "togetherai",
    "llm_model_name": "meta-llama/Meta-Llama-3-8B-Instruct-Lite",
    "embedding_provider": "huggingface",
    "embedding_model_name": "BAAI/bge-large-en-v1.5",
    "embedding_model_dim": null,
    "ollama__host": "192.168.100.

## Set up logger to collect additional info

In [8]:
collect_fp = f"{cfg.notebook_cache_dp}/collect.log"
logger.add(collect_fp, filter=lambda record: "[COLLECT]" in record['message'], mode='w')

1

# Load input data

In [9]:
with open(cfg.data_fp, 'r') as f:
    data = json.load(f)
logger.info(f"[COLLECT] {len(data)=}")
data[0]

2024-07-26 14:53:25.172 | INFO     | __main__:<module>:3 - [COLLECT] len(data)=160


{'title': 'Supercharge your LlamaIndex RAG Pipeline with UpTrain Evaluations',
 'content': "This is a guest post from Uptrain. We are excited to announce the recent integration of LlamaIndex with UpTrain - an open-source LLM evaluation framework to evaluate your RAG pipelines and experiment with different configurations. As an increasing number of companies are graduating their LLM prototypes to production-ready systems, robust evaluations provide a systematic framework to make decisions rather than going with the ‘vibes’. By combining LlamaIndex's flexibility and UpTrain's evaluation framework, developers can experiment with different configurations, fine-tuning their LLM-based applications for optimal performance. About UpTrain UpTrain  [ github  ||  website  ||  docs ] is an open-source platform to evaluate and improve LLM applications. It provides grades for 20+ preconfigured checks (covering language, code, embedding use cases), performs root cause analyses on instances of failure

In [10]:
input_data = data
if ARGS.TESTING:
    input_data = data[:2]
logger.info(f"[COLLECT] {len(input_data)=}")

2024-07-26 14:53:25.199 | INFO     | __main__:<module>:4 - [COLLECT] len(input_data)=160


# Prepare documents

In [11]:
from llama_index.core import Document

documents = []
for record in input_data:
    title = record['title']
    metadata = {
        'title': title,
        'author': record['author'],
        'date': record['date'],
        'tags': ', '.join(record['tags']),
        'url': record['url']
    }
    text = f"{title}\n{record['content']}"
    doc = Document(text=text, metadata=metadata)
    documents.append(doc)

logger.info(f"[COLLECT] {len(documents)=}")

2024-07-26 14:53:25.227 | INFO     | __main__:<module>:17 - [COLLECT] len(documents)=160


# Set up LLM

In [12]:
llm, embed_model = cfg.setup_llm()

In [13]:
print(cfg.llm_cfg.model_dump_json(indent=2))

{
  "llm_provider": "togetherai",
  "llm_model_name": "meta-llama/Meta-Llama-3-8B-Instruct-Lite",
  "embedding_provider": "huggingface",
  "embedding_model_name": "BAAI/bge-large-en-v1.5",
  "embedding_model_dim": 1024,
  "ollama__host": "192.168.100.14",
  "ollama__port": 11434
}


In [14]:
from llama_index.core import Settings
Settings.embed_model = embed_model
Settings.llm = llm

# Vector Store

In [15]:
import qdrant_client
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

from src.run.orchestrator import RunOrchestrator

In [16]:
qdrantdb = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)
aqdrantdb = qdrant_client.AsyncQdrantClient(
    host="localhost",
    port=6333
)

RunOrchestrator.setup_db(cfg, qdrantdb)

db_collection = qdrantdb.get_collection(cfg.db_collection)
vector_store = QdrantVectorStore(
    client=qdrantdb,
    collection_name=cfg.db_collection,
    aclient=aqdrantdb,
    prefer_grpc=True
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

2024-07-26 14:53:33.618 | INFO     | src.run.orchestrator:setup_db:31 - Creating new Qdrant collection huggingface__BAAI_bge_large_en_v1_5__010_remove_title_extractor...
WARNI [llama_index.vector_stores.qdrant.base] Both client and aclient are provided. If using `:memory:` mode, the data between clients is not synced.


# Index Embeddings

In [17]:
import time
import pickle
from llama_index.core.node_parser import SemanticSplitterNodeParser

In [18]:
chunker = SemanticSplitterNodeParser
chunker_cfg = {
    "buffer_size": 1,
    "breakpoint_percentile_threshold": 95,
    "embed_model": embed_model
}

In [19]:
t0 = time.perf_counter()
# TODO: TO understand the differences between points_count and indexed_vector_counts.
# Here indexed_vector_counts = 0
db_collection_count = db_collection.points_count

if db_collection_count > 0 and ARGS.RECREATE_INDEX == False:
    logger.info(f"Loading index from existing DB...")
    with open(cfg.nodes_persist_fp, 'rb') as f:
        logger.info(f"Loading cached `nodes` at {cfg.nodes_persist_fp}...")
        nodes = pickle.load(f)
else:
    logger.info(f"Creating new DB index...")
    from llama_index.core.extractors import TitleExtractor
    from llama_index.core.ingestion import IngestionPipeline, IngestionCache
    
    # create the pipeline with transformations
    pipeline = IngestionPipeline(
        transformations=[
            chunker(**chunker_cfg),
            embed_model,
        ],
        vector_store = vector_store
    )

    num_workers = None
    # Currently setting num_workers leads to error `AttributeError: 'HuggingFaceEmbedding' object has no attribute '_model'`
    # num_workers = os.cpu_count() - 1
    # logger.info(f"Running Ingestion Pipeline with {num_workers=}...")
    logger.info(f"Splitting documents into nodes...")
    nodes = await pipeline.arun(documents=documents, num_workers=num_workers)
    with open(cfg.nodes_persist_fp, 'wb') as f:
        pickle.dump(nodes, f)
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)
t1 = time.perf_counter()

2024-07-26 14:53:34.159 | INFO     | __main__:<module>:12 - Creating new DB index...
2024-07-26 14:53:34.165 | INFO     | __main__:<module>:29 - Splitting documents into nodes...


In [20]:
logger.info(f"Indexing {len(documents)} into VectorStoreIndex took {t1 - t0:,.0f}s")
logger.info(f"[COLLECT] {len(nodes)=}")

2024-07-26 14:59:15.091 | INFO     | __main__:<module>:1 - Indexing 160 into VectorStoreIndex took 341s
2024-07-26 14:59:15.091 | INFO     | __main__:<module>:2 - [COLLECT] len(nodes)=690


# Construct Query Engine

In [21]:
from llama_index.core import get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

from src.features.append_reference.custom_query_engine import ManualAppendReferenceQueryEngine

In [22]:
print(cfg.retrieval_cfg.model_dump_json(indent=2))

{
  "retrieval_top_k": 5,
  "retrieval_similarity_cutoff": null,
  "rerank_top_k": 2,
  "rerank_model_name": "BAAI/bge-reranker-large"
}


In [23]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=cfg.retrieval_cfg.retrieval_top_k,
)

node_postprocessors = []

if cfg.retrieval_cfg.retrieval_similarity_cutoff is not None:
    node_postprocessors.append(SimilarityPostprocessor(similarity_cutoff=cfg.retrieval_cfg.retrieval_similarity_cutoff))

reranker = FlagEmbeddingReranker(model=cfg.retrieval_cfg.rerank_model_name, top_n=cfg.retrieval_cfg.rerank_top_k)
node_postprocessors.append(reranker)

response_synthesizer = get_response_synthesizer()
query_engine = ManualAppendReferenceQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=node_postprocessors,
)

## Test Query Engine

In [24]:
from llama_index.core.response.notebook_utils import (
    display_source_node,
    display_response,
)

In [25]:
question = "How can we address points of failures in RAG pipeline?"
response = query_engine.query(question)
display_response(response, show_source=True, show_metadata=True, show_source_metadata=True)

**`Final Response:`** To address points of failure in RAG pipeline, one approach is to split the query into smaller sub-questions that the LLM can answer more easily. This can be achieved by using the SubQuestionQueryGeneration operator provided by LlamaIndex, which decomposes a question into sub-questions and generates responses for each using an RAG query engine.


Sources:
- [Supercharge your LlamaIndex RAG Pipeline with UpTrain Evaluations](https://www.llamaindex.ai/blog/supercharge-your-llamaindex-rag-pipeline-with-uptrain-evaluations)
- [Introducing Query Pipelines](https://www.llamaindex.ai/blog/introducing-query-pipelines-025dc2bb0537)

---

**`Source Node 1/2`**

**Node ID:** 928a67eb-8b05-46f0-a545-5bceba0e0a5d<br>**Similarity:** 2.259108781814575<br>**Text:** Some of them have multiple parts to them. A good response should be able to answer all the aspect...<br>**Metadata:** {'title': 'Supercharge your LlamaIndex RAG Pipeline with UpTrain Evaluations', 'author': 'Uptrain', 'date': 'Mar 19, 2024', 'tags': 'AI, Evaluation, Rag', 'url': 'https://www.llamaindex.ai/blog/supercharge-your-llamaindex-rag-pipeline-with-uptrain-evaluations'}<br>

---

**`Source Node 2/2`**

**Node ID:** ec9266a0-c40d-4195-9af7-45ef0a896738<br>**Similarity:** -4.099650859832764<br>**Text:** Check out our comprehensive  introduction guide , as well as our  docs page  for more details. Ex...<br>**Metadata:** {'title': 'Introducing Query Pipelines', 'author': 'Jerry Liu', 'date': 'Jan 8, 2024', 'tags': 'Llamaindex, Retrieval Augmented, LLM, AI', 'url': 'https://www.llamaindex.ai/blog/introducing-query-pipelines-025dc2bb0537'}<br>

{'928a67eb-8b05-46f0-a545-5bceba0e0a5d': {'title': 'Supercharge your LlamaIndex RAG Pipeline with UpTrain Evaluations',
  'author': 'Uptrain',
  'date': 'Mar 19, 2024',
  'tags': 'AI, Evaluation, Rag',
  'url': 'https://www.llamaindex.ai/blog/supercharge-your-llamaindex-rag-pipeline-with-uptrain-evaluations'},
 'ec9266a0-c40d-4195-9af7-45ef0a896738': {'title': 'Introducing Query Pipelines',
  'author': 'Jerry Liu',
  'date': 'Jan 8, 2024',
  'tags': 'Llamaindex, Retrieval Augmented, LLM, AI',
  'url': 'https://www.llamaindex.ai/blog/introducing-query-pipelines-025dc2bb0537'}}

# Evaluation

## Retrieval Evaluation

### Synthetic

In [26]:
with open(cfg.nodes_persist_fp, 'rb') as f:
    nodes = pickle.load(f)

In [27]:
from src.run.eval import RetrievalEvaluator

In [28]:
retrieval_evaluator = RetrievalEvaluator()
retrieval_evaluator.generate_synthetic_dataset(cfg, nodes)

2024-07-26 14:59:21.338 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:31 - Creating new retrieval eval dataset at data/010_remove_title_extractor/retrieval_synthetic_eval_dataset.json...
2024-07-26 14:59:21.339 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:35 - Sampling 10 nodes for retrieval evaluation...
2024-07-26 14:59:21.341 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:72 - Creating new synthetic retrieval eval dataset...
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.10s/it]
2024-07-26 14:59:32.329 | INFO     | src.run.eval.retrieval:generate_synthetic_dataset:79 - Persisting synthetic retrieval eval dataset to data/010_remove_title_extractor/retrieval_synthetic_eval_dataset.json...


In [29]:
retrieval_evaluator.retrieval_eval_dataset.queries

{'e9460ed8-0a3f-476e-938a-9a0b3ef2350c': 'How does LlamaCloud improve data handling and OCR accuracy for Scaleport AI across multiple industries?',
 '8f082127-72d3-4eb4-a971-bc37375c03b2': 'What are the key features and enhancements of LlamaCloud, including its integrations with Notion, Slack, Jira, and SharePoint?',
 '199f59db-375d-4ba9-982c-e04bfc229bff': 'How can the alignment and safety of LLMs and LMMs be evaluated?',
 'b20bbd8c-b9f8-4f40-b73a-189af3c8b765': 'What dimensions, other than knowledge and reasoning capabilities, are important to consider when evaluating LLMs and LMMs?',
 'c3fa0f39-45b7-430f-a2da-ce72d59c6b75': 'Who conducted a workshop at the LlamaIndex + Replit Pune Generative AI meetup?',
 '9145c1b1-20e2-447d-baed-333ac298a538': 'Which individuals were involved in the webinar on LLM Challenges in Production?',
 'd9ed22d5-7455-4e57-8dee-cd9531140bf0': 'How does MultiOn manage the action of sending emails through the web browser?',
 'da19fdb4-4fc2-48de-b62e-33fd69db649

In [30]:
retrieval_eval_results_df, retrieval_eval_results_full_df = await retrieval_evaluator.aevaluate(cfg, retriever)

In [31]:
retrieval_eval_results_df

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,top_5_retrieval_eval,0.7,0.616667,0.14,0.7,0.616667,0.216416


In [32]:
retrieval_eval_results_full_df

,query,expected_ids,retrieved_texts,hit_rate,mrr,precision,recall,ap,ndcg
0,How does LlamaCloud improve data handling and ...,[dffd5179-49b0-40dc-b4b1-3e4d9db7073c],[LlamaParse: Outperformed existing OCR soluti...,0.0,0.000000,0.0,0.0,0.000000,0.000000
1,What are the key features and enhancements of ...,[dffd5179-49b0-40dc-b4b1-3e4d9db7073c],[Introducing LlamaCloud and LlamaParse\nToday ...,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,How can the alignment and safety of LLMs and L...,[343d6088-e9ee-47f2-8ab7-5757b895cbc0],[Though studies have shown that strong LLMs ca...,1.0,1.000000,0.2,1.0,1.000000,0.339160
3,"What dimensions, other than knowledge and reas...",[343d6088-e9ee-47f2-8ab7-5757b895cbc0],[Though studies have shown that strong LLMs ca...,1.0,1.000000,0.2,1.0,1.000000,0.339160
4,Who conducted a workshop at the LlamaIndex + R...,[9d825d07-8f33-4146-a108-a31ec39d7e62],[Ravi Theja conducted a workshop at LlamaIn...,1.0,1.000000,0.2,1.0,1.000000,0.339160
5,Which individuals were involved in the webinar...,[9d825d07-8f33-4146-a108-a31ec39d7e62],[Ravi Theja conducted a workshop at LlamaIn...,1.0,1.000000,0.2,1.0,1.000000,0.339160
6,How does MultiOn manage the action of sending ...,[53b3d95c-80a1-46a9-b435-f2d9a278c1a0],"[3. Send Email through MultiOn : Finally, the ...",1.0,1.000000,0.2,1.0,1.000000,0.339160
7,What is the purpose of the integration of Mult...,[53b3d95c-80a1-46a9-b435-f2d9a278c1a0],[Automate online tasks with MultiOn and LlamaI...,1.0,0.500000,0.2,1.0,0.500000,0.213986
8,What is the range of parameters for the large ...,[7964d0e8-2d52-4cd5-8162-572adf498e4b],[This is related to the above statement. A lot...,0.0,0.000000,0.0,0.0,0.000000,0.000000
9,How does the Prometheus model compare to GPT-4...,[7964d0e8-2d52-4cd5-8162-572adf498e4b],[This is in line with the information provided...,1.0,1.000000,0.2,1.0,1.000000,0.339160


#### Error Analysis

In [33]:
retrieval_eval_irrelevance_df = (
    retrieval_eval_results_full_df
    .loc[lambda df: df['hit_rate'].lt(1)]
    .sort_values(['hit_rate', 'mrr', 'precision', 'recall', 'ap', 'ndcg'])
)
retrieval_eval_irrelevance_df

,query,expected_ids,retrieved_texts,hit_rate,mrr,precision,recall,ap,ndcg
0,How does LlamaCloud improve data handling and ...,[dffd5179-49b0-40dc-b4b1-3e4d9db7073c],[LlamaParse: Outperformed existing OCR soluti...,0.0,0.0,0.0,0.0,0.0,0.0
1,What are the key features and enhancements of ...,[dffd5179-49b0-40dc-b4b1-3e4d9db7073c],[Introducing LlamaCloud and LlamaParse\nToday ...,0.0,0.0,0.0,0.0,0.0,0.0
8,What is the range of parameters for the large ...,[7964d0e8-2d52-4cd5-8162-572adf498e4b],[This is related to the above statement. A lot...,0.0,0.0,0.0,0.0,0.0,0.0
14,How does the RetrieverEvaluator module enhance...,[170bc19d-d3db-4b00-95ef-552ef26c491c],[Cohere Reranker. Fine-tuned reranker (Custom ...,0.0,0.0,0.0,0.0,0.0,0.0
16,How does the Advanced Retrieval LlamaPacks sim...,[c38b0ccc-2308-4b80-94bf-3e085f3cb622],[A Cheat Sheet and Some Recipes For Building A...,0.0,0.0,0.0,0.0,0.0,0.0
17,What is the purpose of the OpenAI Cookbook int...,[c38b0ccc-2308-4b80-94bf-3e085f3cb622],[Join us in exploring the depths of RAG system...,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
for i, row in retrieval_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n============Error #{i+1}=============\n\n")
    print(f"Query:\n{row.query}\n")
    expected_contexts = [json.loads(record.payload['_node_content'])['text'] for record in qdrantdb.retrieve(cfg.db_collection, ids=row.expected_ids)]
    expected_contexts = '\n\n'.join(expected_contexts)
    print(f"Expected Contexts:\n{expected_contexts}\n")
    contexts = '\n\n'.join(row.retrieved_texts)
    print(f"Retrieved Contexts:\n{contexts}\n")



============Error #1=============


Query:
How does LlamaCloud improve data handling and OCR accuracy for Scaleport AI across multiple industries?

Expected Contexts:
LlamaIndex Newsletter 2024-07-23
Hello, Llama Followers! 🦙 Welcome to this week’s edition of the LlamaIndex newsletter! We’re thrilled to share some exciting updates about our products, including LlamaCloud, LlamaParse, and LlamaAgents. You’ll also find success stories with LlamaCloud, extensive guides, in-depth tutorials, and information about upcoming hackathons. 🤩  The highlights: LlamaCloud Updates:  New features including LlamaCloud Chat, enhanced Teams collaboration, and expanded integrations with Notion, Slack, Jira, and SharePoint.  Blogpost ,  Tweet . Scaleport AI’s Accelerated Development with LlamaCloud:  Scaleport AI boosts development speed and sales with LlamaCloud and LlamaIndex, improving data handling and OCR accuracy across multiple industries.  Blogpost . Claude Sonnet-3.5 Integration with LlamaParse:

## Response Evaluation

In [35]:
from src.run.eval import ResponseEvaluator

In [36]:
response_evaluator = ResponseEvaluator()

### Synthetic

In [37]:
response_eval_documents, response_synthetic_eval_dataset = response_evaluator.generate_synthetic_dataset(cfg, documents)

2024-07-26 14:59:34.032 | INFO     | src.run.eval.response:generate_synthetic_dataset:39 - Sampling 10 documents for response evaluation...
2024-07-26 14:59:34.033 | INFO     | src.run.eval.response:generate_synthetic_dataset:59 - Creating new response eval dataset at data/010_remove_title_extractor/response_synthetic_eval_dataset.json...
2024-07-26 14:59:34.033 | INFO     | src.run.eval.response:generate_synthetic_dataset:62 - Creating synthetic response eval dataset...


Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.13it/s]
2024-07-26 15:00:25.562 | INFO     | src.run.eval.response:generate_synthetic_dataset:85 - Persisting synthetic response eval dataset at data/010_remove_title_extractor/response_synthetic_eval_dataset.json...


In [38]:
response_synthetic_eval_prediction_dataset = await response_synthetic_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions: 100%|████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:10<00:00,  1.25it/s]


In [39]:
response_synthetic_mean_scores_df, response_synthetic_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_synthetic_eval_dataset,
    response_synthetic_eval_prediction_dataset,
    dataset_name="synthetic",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [01:22<00:00,  2.85s/it]


In [40]:
response_synthetic_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,4.051724
mean_relevancy_score,0.896552
mean_faithfulness_score,0.931034


In [41]:
response_synthetic_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
0,How does the property graph index differ from ...,\nThe property graph index differs from the pr...,1.0,4.5,1.0,"[In the previous integration, the graph was re..."
1,What is the purpose of using the SchemaLLMPath...,\nThe purpose of using the SchemaLLMPathExtrac...,1.0,4.5,1.0,[Introducing the Property Graph Index: A Power...
2,How can parallelization be used to accelerate ...,\nRay Datasets can be used to parallelize buil...,1.0,4.0,1.0,[Build and Scale a Powerful Query Engine with ...
3,How can you adjust the similarity_threshold an...,\nYou can adjust the similarity_threshold and ...,1.0,4.5,1.0,[Note that extracting 250 articles takes about...
4,What methods are used in the custom retriever ...,\nThe custom retriever uses the `entity_extrac...,1.0,4.0,1.0,"[Now, let’s examine our retriever options. At ..."
5,How does the OpenAI Cookbook aim to enhance th...,\nThe OpenAI Cookbook aims to enhance the effe...,1.0,4.5,1.0,[OpenAI Cookbook: Evaluating RAG systems\nWe’r...
6,What tools were used to build the multi-modal ...,\nThe tools used to build the multi-modal prot...,1.0,3.0,1.0,[Here we go over a use case of querying Tesla ...
7,How is the sidebar enhanced to improve credibi...,\nThe sidebar is enhanced to improve credibili...,1.0,4.5,1.0,"[User Interface Next, we craft the sidebar and..."
8,How does the application handle user messages ...,\nThe application handles user messages by lim...,1.0,4.5,0.0,"[Your responses always descriptive. "" \n ..."
9,How has the field of AI and large language mod...,\nThe field of AI and large language models ha...,1.0,4.0,1.0,[Give a followup of each key point with an exp...


#### Error Analysis

In [42]:
response_synthetic_eval_dataset_dict = dict()
for example in response_synthetic_eval_dataset.examples:
    response_synthetic_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [43]:
response_synthetic_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
28,How can one explore the author's Huggingface p...,\nTo explore the author's Hugging Face profile...,0.0,2.5,1.0,"[Additionally, export traces for evaluations a..."
24,What new offering integrates local LLMs and em...,\nLlamaIndex.\n\n\nSources:\n- [Multimodal RAG...,0.0,3.0,1.0,[Image input to LLM. The LLM got two identical...
27,How does a tree index organize data?,\nA tree index organizes data in a hierarchica...,0.0,4.5,1.0,[tree index is a data structure that organizes...
16,What is the length of the 2021 Lyft SEC 10-K d...,"\nAccording to the text, the 2021 SEC 10-K fil...",1.0,2.0,1.0,[It goes into details about the US-China trade...
6,What tools were used to build the multi-modal ...,\nThe tools used to build the multi-modal prot...,1.0,3.0,1.0,[Here we go over a use case of querying Tesla ...
19,What is the purpose of setting up Tonic Validate?,\nThe purpose of setting up Tonic Validate is ...,1.0,3.0,1.0,[As is common in modern software development p...
20,How does Tonic Validate score the LLM responses?,\nThe LLM responses are scored using a set of ...,1.0,3.0,1.0,[I highly recommend utilizing the UI to make v...
2,How can parallelization be used to accelerate ...,\nRay Datasets can be used to parallelize buil...,1.0,4.0,1.0,[Build and Scale a Powerful Query Engine with ...
4,What methods are used in the custom retriever ...,\nThe custom retriever uses the `entity_extrac...,1.0,4.0,1.0,"[Now, let’s examine our retriever options. At ..."
9,How has the field of AI and large language mod...,\nThe field of AI and large language models ha...,1.0,4.0,1.0,[Give a followup of each key point with an exp...


In [44]:
response_synthetic_eval_irrelevance_df = (
    response_synthetic_deep_eval_df
    .loc[lambda df: df['relevancy_score'].lt(1)]
    .sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])
)

for i, row in response_synthetic_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_synthetic_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")



==============Error #1===============


Query:
How can one explore the author's Huggingface profile?

Context:
Additionally, export traces for evaluations and data analysis. All while ensuring your data stays local.  Notebook ,  Tweet . RetrieverEvaluator : new in the library, “RetrieverEvaluator” allows enhanced retrieval evaluations, complementing LLM generation tests. The module supports benchmarking, standard ranking metrics, and synthetic dataset creation for comprehensive retrieval assessments.  Docs ,  Tweet . HuggingFace Embeddings : we added native support for three more Hugging Face embedding models, including the base embeddings wrapper, instructor embeddings, and optimum embeddings in ONNX format.  Docs ,  Tweet . Multi-Document Agents : we’ve introduced v0 experimental support for multi-document agents for advanced QA, beyond typical top-k RAG. It supports diverse queries from single to multiple docs. This foundational version sets the stage for future enhancements like 

### Manually Curated
Ref: https://docs.llamaindex.ai/en/stable/examples/llama_dataset/ragdataset_submission_template/#1c-creating-a-labelledragdataset-from-scratch-with-manually-constructed-examples

In [45]:
response_curated_eval_dataset = response_evaluator.generate_curated_dataset(cfg)

2024-07-26 15:02:12.813 | INFO     | src.run.eval.response:generate_curated_dataset:118 - Persisting curated response eval dataset at data/001/exp_007_semantic_chunking_full_refresh/response_curated_eval_dataset.json...


In [46]:
response_curated_eval_prediction_dataset = await response_curated_eval_dataset.amake_predictions_with(
    predictor=query_engine, batch_size=cfg.batch_size, show_progress=True
)

Batch processing of predictions: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.32s/it]


In [47]:
response_curated_mean_scores_df, response_curated_deep_eval_df = response_evaluator.evaluate_labelled_rag_dataset(
    response_curated_eval_dataset,
    response_curated_eval_prediction_dataset,
    dataset_name="curated",
    judge_model=cfg.eval_cfg.response_eval_llm_model,
    cache_dp=cfg.notebook_cache_dp
)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:10<00:00,  2.62s/it]


In [48]:
response_curated_mean_scores_df

rag,base_rag
metrics,
mean_correctness_score,4.875
mean_relevancy_score,1.000
mean_faithfulness_score,1.000


In [49]:
response_curated_deep_eval_df

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
0,What are key features of llama-agents?,\nDistributed Service-Oriented Architecture: e...,1.0,5.0,1.0,[Introducing llama-agents: A Powerful Framewor...
1,What are the two critical areas of RAG system ...,\nThe two critical areas of RAG system perform...,1.0,5.0,1.0,[OpenAI Cookbook: Evaluating RAG systems\nWe’r...
2,What are the two main metrics used to evaluate...,\nThe two main metrics used to evaluate the pe...,1.0,5.0,1.0,[Boosting RAG: Picking the Best Embedding & Re...
3,How does the MemoryCache project by Mozilla ut...,\nThe MemoryCache project by Mozilla utilizes ...,1.0,4.5,1.0,[Build a ChatGPT with your Private Data using ...


#### Output for seed questions

In [50]:
from src.run.eval.manual_eval_dataset import MANUAL_EVAL_QA

In [51]:
for question, expected_answer in MANUAL_EVAL_QA:
    print(f"\n\nQuestion: {question}\n")
    response = query_engine.query(question)
    display_response(response)



Question: What are key features of llama-agents?



**`Final Response:`** Distributed Service-Oriented Architecture: every agent in LlamaIndex can be its own independently running microservice, orchestrated by a fully customizable LLM-powered control plane that routes and distributes tasks.

Communication via standardized API interfaces: interface between agents using a central control plane orchestrator. Pass messages between agents using a message queue.

Define agentic and explicit orchestration flows: developers have the flexibility to directly define the sequence of interactions between agents, or leave it up to an “agentic orchestrator” that decides which agents are relevant to the task.

Ease of deployment: launch, scale, and monitor each agent and your control plane independently.

Scalability and resource management: use our built-in observability tools to monitor the quality and performance of the system and each individual agent service.


Sources:
- [Introducing llama-agents: A Powerful Framework for Building Production Multi-Agent AI Systems](https://www.llamaindex.ai/blog/introducing-llama-agents-a-powerful-framework-for-building-production-multi-agent-ai-systems)
- [LlamaIndex Newsletter 2024-07-02](https://www.llamaindex.ai/blog/llamaindex-newsletter-2024-07-02)



Question: What are the two critical areas of RAG system performance that are assessed in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook?



**`Final Response:`** The two critical areas of RAG system performance that are assessed in the 'Evaluating RAG with LlamaIndex' section of the OpenAI Cookbook are the Retrieval System and Response Generation.


Sources:
- [OpenAI Cookbook: Evaluating RAG systems](https://www.llamaindex.ai/blog/openai-cookbook-evaluating-rag-systems-fe393c61fb93)



Question: What are the two main metrics used to evaluate the performance of the different rerankers in the RAG system?



**`Final Response:`** The two main metrics used to evaluate the performance of the different rerankers in the RAG system are Hit Rate and Mean Reciprocal Rank (MRR).


Sources:
- [Boosting RAG: Picking the Best Embedding & Reranker models](https://www.llamaindex.ai/blog/boosting-rag-picking-the-best-embedding-reranker-models-42d079022e83)



Question: How does the MemoryCache project by Mozilla utilize PrivateGPT_AI and LlamaIndex to enhance personal knowledge management while maintaining privacy? Provide a brief overview of the project and its key features.



**`Final Response:`** The MemoryCache project by Mozilla utilizes PrivateGPT_AI and LlamaIndex to enhance personal knowledge management while maintaining privacy by integrating private knowledge sources, such as company data, into Large Language Models (LLMs) like ChatGPT. This integration enables users to access and utilize their private knowledge sources, like internal documents, to generate more accurate and relevant responses.


Sources:
- [Build a ChatGPT with your Private Data using LlamaIndex and MongoDB](https://www.llamaindex.ai/blog/build-a-chatgpt-with-your-private-data-using-llamaindex-and-mongodb-b09850eb154c)

#### Error Analysis

In [52]:
response_curated_eval_dataset_dict = dict()
for example in response_curated_eval_dataset.examples:
    response_curated_eval_dataset_dict[example.query] = {
        "reference_answer": example.reference_answer,
        "reference_contexts": example.reference_contexts,
    }

In [53]:
response_curated_deep_eval_df.sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])

,query,answer,relevancy_score,correctness_score,faithfulness_score,contexts
3,How does the MemoryCache project by Mozilla ut...,\nThe MemoryCache project by Mozilla utilizes ...,1.0,4.5,1.0,[Build a ChatGPT with your Private Data using ...
0,What are key features of llama-agents?,\nDistributed Service-Oriented Architecture: e...,1.0,5.0,1.0,[Introducing llama-agents: A Powerful Framewor...
1,What are the two critical areas of RAG system ...,\nThe two critical areas of RAG system perform...,1.0,5.0,1.0,[OpenAI Cookbook: Evaluating RAG systems\nWe’r...
2,What are the two main metrics used to evaluate...,\nThe two main metrics used to evaluate the pe...,1.0,5.0,1.0,[Boosting RAG: Picking the Best Embedding & Re...


In [54]:
response_curated_eval_irrelevance_df = (
    response_curated_deep_eval_df
    .loc[lambda df: df['relevancy_score'].lt(1)]
    .sort_values(['relevancy_score', 'correctness_score', 'faithfulness_score'])
)

for i, row in response_curated_eval_irrelevance_df.reset_index(drop=True).iterrows():
    print(f"\n\n==============Error #{i+1}===============\n\n")
    print(f"Query:\n{row.query}\n")
    contexts = '\n\n'.join(row.contexts)
    print(f"Context:\n{contexts}\n")
    print(f"Answer:\n{row.answer}\n----\n")
    expected = response_curated_eval_dataset_dict.get(row.query)
    if not expected:
        logger.error(f"Could not find query {row.query} in synthetic_response_eval_dataset_dict!")
        continue
    expected_answer = expected['reference_answer']
    print(f"Expected Answer:\n{expected_answer}\n")
    expected_contexts = expected['reference_contexts']
    print(f"Expected Contexts:\n{expected_contexts}\n")

# Persist run metadata

In [55]:
from src.run.utils import parse_collect_log, flatten_dict

In [56]:
collect_info = parse_collect_log(collect_fp)

In [57]:
if ARGS.LOG_TO_MLFLOW:
    import mlflow

    logger.info("Logging [COLLECT] info to MLflow...")
    mlflow.log_params(collect_info)
    logger.info("Logging config to MLflow...")
    mlflow.log_params(flatten_dict(cfg.model_dump(), "cfg", sep='.'))
    logger.info(f"Logging Retrieval Synthetic Eval Results to MLflow...")
    retrieval_evaluator.log_to_mlflow(cfg)
    logger.info(f"Logging Response Eval Results to MLflow...")
    response_evaluator.log_to_mlflow(
        cfg,
        'synthetic',
        response_synthetic_mean_scores_df,
        response_synthetic_deep_eval_df
    )
    response_evaluator.log_to_mlflow(
        cfg,
        'curated',
        response_curated_mean_scores_df,
        response_curated_deep_eval_df
    )

2024-07-26 15:02:36.568 | INFO     | __main__:<module>:4 - Logging [COLLECT] info to MLflow...
2024-07-26 15:02:36.593 | INFO     | __main__:<module>:6 - Logging config to MLflow...
/home/dvquys/frostmourne/study/vietai-genai03/assignment1/.venv/lib/python3.11/site-packages/pydantic/main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  Expected `int` but got `str` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
2024-07-26 15:02:36.640 | INFO     | __main__:<module>:8 - Logging Retrieval Synthetic Eval Results to MLflow...
2024-07-26 15:02:36.913 | INFO     | __main__:<module>:10 - 

# Clean up

In [58]:
if ARGS.LOG_TO_MLFLOW:
    mlflow.end_run()